In [1]:
from IPython.display import display
from math import ceil, sqrt
import numpy as np 
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import statsmodels.api as sm

from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import lag_plot
from statsmodels.tsa.stattools import adfuller

# colors generator
from itertools import cycle
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# # Algorithms
# from sklearn.cluster import KMeans
# from sklearn.metrics import pairwise_distances_argmin_min
# from sklearn.decomposition import PCA
# from tslearn.barycenters import dtw_barycenter_averaging
# Modeling
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# from statsmodels.tsa.arima_model import ARIMA
# import pmdarima as pm
from sklearn.model_selection import ParameterGrid
# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
# assert reproducibility
import random
random.seed(7)
np.random.seed(7)
print("setup complete")

setup complete


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
month_in_year = 12
month_quarter =4
window1 = 3
window2 = 6
window3 = 12

def add_vars(y):
    df = y.copy()
    y_name = df.columns[0]
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['quarter'] = df.index.quarter
    df['month_sin'] = np.sin(2*np.pi*df['month']/month_in_year)
    df['month_cos'] = np.cos(2*np.pi*df['month']/month_in_year)
    df['quarter_sin'] = np.sin(2*np.pi*df['quarter']/month_quarter)
    df['quarter_cos'] = np.cos(2*np.pi*df['quarter']/month_quarter)
    
    for i in range(1, 13):
        df[f"lag{i}"] = df[y_name].shift(i)
    df["Date"] = y[[y_name]].index
    
    df.reset_index(drop=True, inplace=True)

    df_rolled_3d = df[[y_name]].rolling(window=window1, min_periods=0)
    df_rolled_7d = df[[y_name]].rolling(window=window2, min_periods=0)
    df_rolled_30d = df[[y_name]].rolling(window=window3, min_periods=0)

    df_mean_3d = df_rolled_3d.mean().shift(1).reset_index().astype(np.float32)
    df_mean_7d = df_rolled_7d.mean().shift(1).reset_index().astype(np.float32)
    df_mean_30d = df_rolled_30d.mean().shift(1).reset_index().astype(np.float32)

    df_std_3d = df_rolled_3d.std().shift(1).reset_index().astype(np.float32)
    df_std_7d = df_rolled_7d.std().shift(1).reset_index().astype(np.float32)
    df_std_30d = df_rolled_30d.std().shift(1).reset_index().astype(np.float32)

    df[f"{y_name}_mean_lag{window1}"] = df_mean_3d[y_name]
    df[f"{y_name}_mean_lag{window2}"] = df_mean_7d[y_name]
    df[f"{y_name}_mean_lag{window3}"] = df_mean_30d[y_name]

    df[f"{y_name}_std_lag{window1}"] = df_std_3d[y_name]
    df[f"{y_name}_std_lag{window2}"] = df_std_7d[y_name]
    df[f"{y_name}_std_lag{window3}"] = df_std_30d[y_name]

    df.fillna(df.mean(), inplace=True)

    df.set_index("Date", drop=False, inplace=True)
    return df

In [4]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [5]:
def split_serie(df):
    """
        Add vars & return X_train, X_test, y_train, y_test
    """
    ts = df.copy()
    ts = add_vars(ts)
    y_name = ts.columns[0]
    y = ts.dropna()[y_name]
    X = ts.dropna().drop([y_name, 'Date'], axis=1)
    
    return timeseries_train_test_split(X, y, test_size=0.15)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
def prepare_data(df):
    """
        Add vars & return X_train_scaled, X_test_scaled, y_train, y_test
    """
    X_train, X_test, y_train, y_test = split_serie(df)
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = pd.DataFrame(X_train_scaled)
    X_train_scaled.columns = X_train.columns
    X_test_scaled = pd.DataFrame(X_test_scaled)
    X_test_scaled.columns = X_train.columns
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [8]:
from sklearn.model_selection import TimeSeriesSplit

In [9]:
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import cross_val_score

# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

In [10]:
def plotModelResults(models, X_train, X_test, y_train, y_test,\
                     plot_intervals=False, plot_anomalies=False, axe = False):
    """
        Plots modelled vs fact values, prediction intervals and anomalies
        models = [["ridge": mod_r], ["lasso": mod_l]]
    """
    
    errors = [0]*len(models)
    predictions = [0]*len(models)
    for i, model in enumerate(models) :
        predictions[i] = model[1].predict(X_test)
        errors[i] = mean_absolute_percentage_error(predictions[i], y_test)

    min_index = errors.index(min(errors))
    error = errors[min_index]
    prediction = predictions[min_index]
    winner = models[min_index][0]
    model = models[min_index][1]
    
    index_of_pred = y_test.index
    pred_series = pd.Series(prediction, index=index_of_pred)
    
    if not axe : 
        axe = plt
        axe.figure(figsize=(15, 7))
    axe.plot(pred_series, "g", label="prediction", linewidth=2.0)
    axe.plot(pd.concat([y_train, y_test]), label="actual", linewidth=2.0)
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train, 
                                    cv=tscv, 
                                    scoring="neg_mean_absolute_error")
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = pred_series - (mae + scale * deviation)
        upper = pred_series + (mae + scale * deviation)
        
        axe.plot(lower, "r--", label="upper bond / lower bond", alpha=0.5)
        axe.plot(upper, "r--", alpha=0.5)
        
        if plot_anomalies:
            anomalies = np.array([np.NaN]*len(y_test))
            anomalies[y_test<lower] = y_test[y_test<lower]
            anomalies[y_test>upper] = y_test[y_test>upper]
            anomalies_series = pd.Series(anomalies, index=index_of_pred)
            if ~np.isnan(anomalies).all() : axe.plot(anomalies_series, "o", markersize=10, label = "Anomalies")
    
    f = axe.title if axe == plt else axe.set_title
    f(f"{y_test.name},  model : {winner},  Mean absolute percentage error {error:.2f}%")
    axe.axvspan(index_of_pred[0], index_of_pred[-1],  color=sns.xkcd_rgb['grey'], alpha=0.2)
    axe.legend(loc="best")
    
def plotCoefficients(model, X_train):
    """
        Plots sorted coefficient values of the model
    """
    
    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ["coef"]
    coefs["abs"] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by="abs", ascending=False).drop(["abs"], axis=1)
    
    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');

### les fct aprés xgb

In [11]:
from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

def objective(space):
    # Instantiate the classifier
    clf = XGBRegressor(n_estimators =1000,colsample_bytree=space['colsample_bytree'],
                           learning_rate = .3,
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'],
                           reg_lambda = space['reg_lambda'])
    
    eval_set  = [(X_train_scaled, y_train), (X_test_scaled, y_test)]
    
    # Fit the classsifier
    clf.fit(X_train_scaled, y_train,
            eval_set=eval_set, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)
    
    # Predict on Cross Validation data
    pred = clf.predict(X_test_scaled)
    
    # Calculate our Metric - accuracy
    accuracy = accuracy_score(y_test, pred>0.5)
# return needs to be in this below format. We use negative of accuracy since we want to maximize it.
    return {'loss': -accuracy, 'status': STATUS_OK }

ModuleNotFoundError: No module named 'hyperopt'

In [ ]:
space ={'max_depth': hp.quniform("max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('min_child_weight', 1, 10, 1),
        'subsample': hp.uniform ('subsample', 0.7, 1),
        'gamma' : hp.uniform ('gamma', 0.1,0.5),
        'colsample_bytree' : hp.uniform ('colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('reg_lambda', 0,1)
    }

In [ ]:
def auto_features_forecast(series, models, test_size=0.15):
    fig, axs = plt.subplots(5, 2, figsize=(20, 20))
    axs = axs.flatten()
    for i, col in enumerate(series.columns):
        global X_train_scaled, y_train, X_test_scaled, y_test
        X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(series[[col]])
        for j in range(len(models)) :
            if "xgboost" in models[j][0] :
                if "tune" in models[j][0] :
                    trials = Trials()
                    best = fmin(fn=objective,
                                space=space,
                                algo=tpe.suggest,
                                max_evals=10,
                                trials=trials)                
                    best['max_depth'] = int(best['max_depth'])

                    models[j][1] = XGBRegressor(n_estimators =1000, **best)
                    models[j][1].fit(X_train_scaled, y_train, \
                              eval_set=[(X_train_scaled, y_train),(X_test_scaled, y_test)],\
                              early_stopping_rounds=50, verbose=False)
                else :
                    models[j][1].fit(X_train_scaled, y_train, \
                              eval_set=[(X_train_scaled, y_train),(X_test_scaled, y_test)],\
                              early_stopping_rounds=50, verbose=False)
            else : 
                models[j][1].fit(X_train_scaled, y_train)
        plotModelResults(models, 
                         X_train=X_train_scaled, 
                         X_test=X_test_scaled, 
                         y_train=y_train,
                         y_test=y_test,
                         plot_intervals=True, plot_anomalies=True, axe = axs[i])

In [ ]:
sales_s = pd.read_pickle("sales_s.pkl")
sales_s.head()

Pour la démonstration de la demarche suivie dans l'apprentissage supervisé, on a selectionné aléatoirement un produit (P_64).


In [ ]:
sales_s_p64 = sales_s[['P_64']]
sales_s_p64.head()


On a commencé par implémenter le concept de "feature engineering" ou on a généré les variables nécessaires pour entrainer les modèles d'apprentissage.

- Variables de temps : année, mois, trimestre
- Valeurs retardées des séries temporelles
- Statistiques : la moyenne et l'écart type des valeurs retardées. 

titre : Visualisation des variables cycliques 

Les variables temporelles tels que le mois et le trimestre sont cycliques. Par exemple, le mois oscille entre 1 et 12 pour chaque année. Alors que la différence entre chaque mois augmente de 1 au cours de l'année, entre deux ans, la caractéristique du mois passe de 12 (décembre) à 1 (janvier). Il en résulte une différence de -11, ce qui peut dérouter beaucoup de modèles.


Pour résoudre ce problème, on a effectué une transformation par la fonction cosinus après avoir normalisé les variables entre 0 et 2π, ce qui correspond à un cycle de cosinus, cette solution ne régle pas le problème complétement car deux valeurs différentes peuvent avoir la même image par la fonction cosinus. La meilleure façon de résoudre ce nouveau problème était d'ajouter une autre information cyclique pour distinguer deux temps avec des valeurs de cosinus identiques, il s'agit de la fonction sinus. Nous pourrions le considérer comme un système de coordonnées à deux axes.

In [ ]:
demo = sales_s_p64.copy()
demo['year'] = demo.index.year
demo['month'] = demo.index.month
demo['quarter'] = demo.index.quarter
demo.tail()

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 3))
sns.lineplot(x=demo.index, y=demo['quarter'], color='dodgerblue')
ax.set_xlim(["2014-07-31", "2019-09-30"])
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 3))
sns.lineplot(x=demo.index, y=demo['month'], color='dodgerblue')
ax.set_xlim(["2014-07-31", "2019-09-30"])
plt.show()

In [ ]:
month_in_year = 12
demo['month_sin'] = np.sin(2*np.pi*demo['month']/month_in_year)
demo['month_cos'] = np.cos(2*np.pi*demo['month']/month_in_year)
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
sns.scatterplot(x=demo.month_sin, y=demo.month_cos, color='dodgerblue')
plt.show()

In [ ]:
month_quarter =4
demo['quarter_sin'] = np.sin(2*np.pi*demo['quarter']/month_quarter)
demo['quarter_cos'] = np.cos(2*np.pi*demo['quarter']/month_quarter)
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
sns.scatterplot(x=demo.quarter_sin, y=demo.quarter_cos, color='dodgerblue')
plt.show()

In [ ]:
demo.head()

Comme indiqué précédemment, on a utilisé les variables de la série décalées pour beneficier de l'autocorrélation existante dans la série. Pour prendre en compte l'effet de la saisonnalité, on a considéré 12 retards.

In [ ]:
for i in range(1, 13):
    demo[f"lag{i}"] = demo.P_64.shift(i)

In [ ]:
demo2 = demo.copy()
demo2["Date"] = demo[['P_64']].index

On a généré les moyennes mobiles et ecart-types mobiles (trimestrielle, semestrielle et annuelle) des valeurs décalées afin de détecter la tendance globale des données.

In [ ]:
demo2.reset_index(drop=True, inplace=True)
window1 = 3
window2 = 6
window3 = 12

df_rolled_3d = demo2[["P_64"]].rolling(window=window1, min_periods=0)
df_rolled_7d = demo2[["P_64"]].rolling(window=window2, min_periods=0)
df_rolled_30d = demo2[["P_64"]].rolling(window=window3, min_periods=0)

df_mean_3d = df_rolled_3d.mean().shift(1).reset_index().astype(np.float32)
df_mean_7d = df_rolled_7d.mean().shift(1).reset_index().astype(np.float32)
df_mean_30d = df_rolled_30d.mean().shift(1).reset_index().astype(np.float32)

df_std_3d = df_rolled_3d.std().shift(1).reset_index().astype(np.float32)
df_std_7d = df_rolled_7d.std().shift(1).reset_index().astype(np.float32)
df_std_30d = df_rolled_30d.std().shift(1).reset_index().astype(np.float32)

demo2[f"{'P_64'}_mean_lag{window1}"] = df_mean_3d["P_64"]
demo2[f"{'P_64'}_mean_lag{window2}"] = df_mean_7d["P_64"]
demo2[f"{'P_64'}_mean_lag{window3}"] = df_mean_30d["P_64"]

demo2[f"{'P_64'}_std_lag{window1}"] = df_std_3d["P_64"]
demo2[f"{'P_64'}_std_lag{window2}"] = df_std_7d["P_64"]
demo2[f"{'P_64'}_std_lag{window3}"] = df_std_30d["P_64"]

demo2.fillna(demo2.mean(), inplace=True)

demo2.set_index("Date", drop=False, inplace=True)
demo2.head()

Pour la validation des modèles on a divisé le jeu de données en deux parties : la premiere partie pour l'entrainement du modèle (train set) et le reste (la derniere année) est reservé pour l'évaluation.

On a ajusté un modèle de regression linéaire multiple sur l'ensemble des variables et on a obtenu une erreur (MAPE) de 5.29% sur le test set.

In [ ]:
X_train, X_test, y_train, y_test = split_serie(sales_s_p64)

In [ ]:
X_train_scaled, X_test_scaled, y_train, y_test = prepare_data(sales_s_p64)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
plotModelResults([["Linear Regression", lr]], X_train=X_train, X_test=X_test, \
                 y_train=y_train, y_test=y_test, plot_intervals=True)
plotCoefficients(lr, X_train)

Ce premier modèle présente des défauts

La régression linéaire a donné des résultats assez biens. Cependant ce modèle simple présente des défauts,certaines variables explicatives sont fortement corrélées et toutes les variables ne sont pas également saines,certaines peuvent entraîner un surapprentissage tandis que d'autres doivent être supprimées.

- le modèle n'arrive pas à utiliser toute l'information disponible aux varibales explicatives ??

D'autre part, puisque on a différentes échelles dans les variables, celles ci doivent etre transformées en une même échelle pour explorer l'importance des caractéristiques et, plus tard, la régularisation.

In [ ]:
cg = sns.clustermap(X_train_scaled.corr(), figsize=(10, 10), cbar_pos=(.1, .1, .03, .65))
cg.ax_row_dendrogram.set_visible(False)
cg.ax_col_dendrogram.set_visible(False)
plt.show()

Pour une meilleure optimisation des variables explicatives, on a  appliqué la régularisation. Deux des modèles de régression avec régularisation les plus populaires sont les régressions Ridge et Lasso. Ils ajoutent tous deux des contraintes supplémentaires à notre fonction de perte.

Dans le cas de la régression Ridge, ces contraintes sont la somme des carrés des coefficients multipliée par le coefficient de régularisation. Plus le coefficient d'une caractéristique est grand, plus notre perte sera importante. Par conséquent, nous essaierons d'optimiser le modèle tout en gardant les coefficients assez bas.

À la suite de cette régularisation L2, nous aurons un biais plus élevé et une variance plus faible, donc le modèle se généralisera mieux.

Le deuxième modèle de régression, la régression Lasso, ajoute à la fonction de perte, non pas des carrés, mais des valeurs absolues des coefficients. Par conséquent, au cours du processus d'optimisation, les coefficients des caractéristiques sans importance peuvent devenir des zéros, ce qui permet une sélection automatisée des caractéristiques. Ce type de régularisation est appelé L1.

In [ ]:
ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)
plotModelResults([["ridge", ridge]], 
                 X_train=X_train_scaled, 
                 X_test=X_test_scaled,
                 y_train=y_train,
                 y_test=y_test,
                 plot_intervals=True, plot_anomalies=True)
plotCoefficients(ridge, X_train_scaled)

La figure ci-dessous représente les prévisions pour chaque produit par le modèle qui minimise l'erreur (MAPE) parmis les deux modèles de régularisation.

In [ ]:
ridge = RidgeCV(cv=tscv)
lasso = LassoCV(cv=tscv)
auto_features_forecast(sales_s, [["Lasso", lasso], ["Ridge", ridge]])

In [ ]:
from xgboost import XGBRegressor, plot_importance, plot_tree

In [ ]:
best = {'colsample_bytree': 0.7855436372368847,
 'gamma': 0.2715026337470021,
 'max_depth': 11,
 'min_child_weight': 1.0,
 'reg_lambda': 0.358974472027333,
 'subsample': 0.8129759325416941}

In [ ]:
reg = XGBRegressor(n_estimators =1000, **best)
reg.fit(X_train_scaled, y_train,
        eval_set=[(X_train_scaled, y_train), (X_test_scaled, y_test)],
        early_stopping_rounds=50,
       verbose=False)

In [ ]:
prediction = reg.predict(X_test_scaled)

In [ ]:
index_of_pred = y_test.index
pred_series = pd.Series(prediction, index=index_of_pred)
plt.plot(pred_series, "g", label="prediction", linewidth=2.0)
plt.plot(pd.concat([y_train, y_test]), label="actual", linewidth=2.0)

In [ ]:
plotModelResults([["xgboost", reg]], 
                         X_train=X_train_scaled, 
                         X_test=X_test_scaled, 
                         y_train=y_train,
                         y_test=y_test,
                         plot_intervals=True, plot_anomalies=True)

In [ ]:
_ = plot_importance(reg, height=0.9)

In [ ]:
auto_features_forecast(sales_s, [["tuned_xgboost", 0], ["default_xgboost", XGBRegressor(n_estimators =1000)]])

In [ ]:
auto_features_forecast(sales_s, [["tuned_xgboost", 0], ["default_xgboost", XGBRegressor(n_estimators =1000)],\
                                 ["Lasso", lasso], ["Ridge", ridge]])